In [1]:
import geopandas as gpd
import pandas as pd

In [470]:
##L'heure et la date de la requête : 
import time
import datetime
#On regarde l'heure et la date de la requête
time.strftime('%Y-%m-%d %H:%M:%S', time.localtime()) #Date et heure en entier
Date = time.strftime('%Y-%m-%d', time.localtime())
Time = time.strftime('%H:%M', time.localtime()) #Heure uniquement
DateTime = time.strftime('%Y-%m-%dT22:00:00', time.localtime()) 
# Pour faciliter ici, je n'ai pas arrondi les minutes, mais rogné
#Je suis partie du postulat que pour nos données, la précision n'était pas ce qui était  le principal à rechercher
print('Voic la date de la recherche : ', Date)
print("Voici l'heure de la recherche :", Time)
print(DateTime, 'heure')

Voic la date de la recherche :  2020-12-08
Voici l'heure de la recherche : 21:56
2020-12-08T22:00:00 heure


On prélève les données géopandas sur le site : https://www.data.gouv.fr/fr/datasets/prevision-meteo-ile-de-france-arome/#_

## la météo en temps réel

In [490]:
meteo_gpd = 'https://www.data.gouv.fr/fr/datasets/r/8e834f69-0b43-439a-87bd-87ca80c84cd7'
meteo_idf = gpd.read_file(meteo_gpd) 
#Cette base de donnée, seule gratuite qui correspond à ce que l'on veut, donne les prévisions pour les prochaines 36h. 
#On peut planifier la météo pour le trajet à quelques heures près dans la journée qui arrive
#Donne la prévision qui a été faite la veille à 13h.

D'après la documentation, le total_water_precipitation correspond au cumul (depuis le début de la simulation) de la somme des flux au sol des précipitations liquides et solides (EAU + NEIGE + GRAUPEL) en kg/m-2
 la température : C’est la température (voir T) diagnostiquée à 2m de hauteur au dessus du relief . Ce paramètre peut a
priori être comparé aux mesures faites in situ (stations d’observations humaines ou automatiques).

In [491]:
meteo_idf.head(2)

,surface_net_thermal_radiation,downward_short_wave_radiation_flux,2_metre_temperature,minimum_temperature_at_2_metres,surface_net_solar_radiation,timestamp,surface_latent_heat_flux,wind_speed,total_water_precipitation,relative_humidity,surface_sensible_heat_flux,forecast,maximum_temperature_at_2_metres,geometry
0,-847709.0,1748221.25,2.887476,2.889429,1484716.25,2020-12-08T13:00:00,-625017.0,0.838349,0.0,92.738327,-302137.0,2020-12-08T18:00:00,3.608423,POINT (2.17500 49.15000)
1,-884292.0,1748221.25,2.841699,2.781152,1484716.25,2020-12-08T13:00:00,-625588.0,0.820849,0.0,93.110840,-298191.0,2020-12-08T19:00:00,2.835229,POINT (2.17500 49.15000)


In [492]:
meteo_idf = meteo_idf[['timestamp', 'forecast', '2_metre_temperature', 'total_water_precipitation',  'geometry']]

In [493]:
meteo_idf["duration"] = pd.to_datetime(meteo_idf["forecast"]) - pd.to_datetime(meteo_idf["timestamp"])
meteo_idf["duration"] = meteo_idf["duration"].astype("timedelta64[m]")
#Différence en minutes entre la prévision et l'heure prévue
meteo_idf = meteo_idf.cx[2.35,48.85] #On n'a que les prévisions pour le centre de Paris, c'est déjà bien
meteo_idf = meteo_idf.sort_values('forecast').sample(15) # On a bien toutes les heures les unes après les autres
meteo_idf

,timestamp,forecast,2_metre_temperature,total_water_precipitation,geometry,duration
19446,2020-12-08T13:00:00,2020-12-09T01:00:00,2.915125,0.000000,POINT (2.35000 48.85000),720.0
18339,2020-12-08T13:00:00,2020-12-08T18:00:00,4.358179,0.000000,POINT (2.35000 48.85000),300.0
19445,2020-12-08T13:00:00,2020-12-08T16:00:00,5.334100,0.000000,POINT (2.35000 48.85000),180.0
18336,2020-12-08T13:00:00,2020-12-08T17:00:00,4.789636,0.000000,POINT (2.35000 48.85000),240.0
17975,2020-12-08T13:00:00,2020-12-10T01:00:00,3.554773,0.113281,POINT (2.35000 48.85000),2160.0
17973,2020-12-08T13:00:00,2020-12-09T14:00:00,4.096948,0.060547,POINT (2.35000 48.85000),1500.0
19449,2020-12-08T13:00:00,2020-12-09T12:00:00,2.676355,0.035156,POINT (2.35000 48.85000),1380.0
17758,2020-12-08T13:00:00,2020-12-09T08:00:00,0.074701,0.011719,POINT (2.35000 48.85000),1140.0
18345,2020-12-08T13:00:00,2020-12-08T22:00:00,4.402551,0.000000,POINT (2.35000 48.85000),540.0
18357,2020-12-08T13:00:00,2020-12-09T23:00:00,3.474176,0.113281,POINT (2.35000 48.85000),2040.0


On pourrait faire un prolongement, en regardant exactement la localisation du point de départ et d'arrivée, mais bon, déjà, j'essyae avec ça

Je convertis les données de précipitations qui sont en kg/m-2 en données de précipitations :

#En une heure, on compte total-water_precipitation * 3600 / le nombre de minutes depuis timestamp

Je sais que : Mesures de la pluviométrie :  La mesure de la pluie, appelée pluviométrie, se fait avec un simple appareil nommé le pluviomètre. Cette mesure correspond à la hauteur d'eau recueillie sur une surface plane. Elle s'exprime en millimètres, et parfois en litres par mètre carré (1 litre/m2 = 1 mm). On sépare l'intensité de la pluie en pluie faible (trace à 2 mm/h), modérée (2 mm/h à 7,6 mm/h) et forte (plus de 7,6 mm/h)

In [495]:
meteo_idf["precipitations mm"] = ((meteo_idf["total_water_precipitation"])/(meteo_idf["duration"]))*60
##A vérifier
meteo_idf.sort_values('forecast')

,timestamp,forecast,2_metre_temperature,total_water_precipitation,geometry,duration,precipitations mm
19445,2020-12-08T13:00:00,2020-12-08T16:00:00,5.334100,0.000000,POINT (2.35000 48.85000),180.0,0.000000
18336,2020-12-08T13:00:00,2020-12-08T17:00:00,4.789636,0.000000,POINT (2.35000 48.85000),240.0,0.000000
18339,2020-12-08T13:00:00,2020-12-08T18:00:00,4.358179,0.000000,POINT (2.35000 48.85000),300.0,0.000000
18345,2020-12-08T13:00:00,2020-12-08T22:00:00,4.402551,0.000000,POINT (2.35000 48.85000),540.0,0.000000
19446,2020-12-08T13:00:00,2020-12-09T01:00:00,2.915125,0.000000,POINT (2.35000 48.85000),720.0,0.000000
17757,2020-12-08T13:00:00,2020-12-09T03:00:00,2.731287,0.001953,POINT (2.35000 48.85000),840.0,0.000140
18681,2020-12-08T13:00:00,2020-12-09T06:00:00,-0.724646,0.005859,POINT (2.35000 48.85000),1020.0,0.000345
17758,2020-12-08T13:00:00,2020-12-09T08:00:00,0.074701,0.011719,POINT (2.35000 48.85000),1140.0,0.000617
18685,2020-12-08T13:00:00,2020-12-09T10:00:00,1.521326,0.011719,POINT (2.35000 48.85000),1260.0,0.000558
19449,2020-12-08T13:00:00,2020-12-09T12:00:00,2.676355,0.035156,POINT (2.35000 48.85000),1380.0,0.001529


In [496]:
DateTime

'2020-12-08T22:00:00'

In [497]:
meteo_idf.loc[meteo_idf['forecast'] == DateTime]
#On a la température, et le total d'eau depuis la date du timestamp en fonction du temps

,timestamp,forecast,2_metre_temperature,total_water_precipitation,geometry,duration,precipitations mm
18345,2020-12-08T13:00:00,2020-12-08T22:00:00,4.402551,0.0,POINT (2.35000 48.85000),540.0,0.0


## Modifications de la base accident vélo
Le but est de regrouper les informations

In [ ]:
accident_velo = pd.read_csv('https://www.data.gouv.fr/fr/datasets/r/3d5f2317-5afd-4a9f-a9c5-bd4fe0113f39')
accident_velo_idf = accident_velo[accident_velo['departement'].isin(['75','77','78','91','92','93','94','95'])]
# Il y a un problème avec les latitudes et les longitudes nulles et les quelques erreurs

In [485]:
accident_velo_idf_meteo = accident_velo_idf[['identifiant accident','date','mois', 'jour', 'heure', 'departement','luminosite', 'conditions atmosperiques', 'etat surface', 'sexe', 'age', 'gravite accident', 'commune', ]]
accident_velo_idf_meteo

,identifiant accident,date,mois,jour,heure,departement,luminosite,conditions atmosperiques,etat surface,sexe,age,gravite accident,commune
58,200500001606,2005-01-13,01 - janvier,3 - jeudi,11.0,78,Jour,Normale,normale,M,61-62,2 - Blessé hospitalisé,78230
61,200500001930,2005-01-15,01 - janvier,5 - samedi,15.0,95,Jour,Normale,normale,M,58-59,0 - Indemne,95042
221,200500004591,2005-01-04,01 - janvier,1 - mardi,17.0,75,Nuit avec éclairage public allumé,Normale,normale,M,36-37,1 - Blessé léger,75001
222,200500004597,2005-01-03,01 - janvier,0 - lundi,9.0,75,Jour,Normale,normale,M,26-27,0 - Indemne,75005
223,200500004600,2005-01-03,01 - janvier,0 - lundi,17.0,75,Nuit avec éclairage public allumé,Normale,normale,M,31-32,0 - Indemne,75019
...,...,...,...,...,...,...,...,...,...,...,...,...,...
65531,201800050291,2018-07-10,07 - juillet,1 - mardi,9.0,75,Jour,Normale,normale,F,58-59,2 - Blessé hospitalisé,75101
65532,201800050337,2018-11-17,11 - novembre,5 - samedi,12.0,75,Jour,Normale,normale,M,59-60,1 - Blessé léger,75101
65533,201800050355,2018-02-01,02 - février,3 - jeudi,15.0,75,Jour,Temps couvert,mouillée,M,31-32,1 - Blessé léger,75102
65534,201800050359,2018-02-12,02 - février,0 - lundi,9.0,75,Jour,Normale,normale,F,29-30,1 - Blessé léger,75101


Le but, c'est d'utiliser les 18 000 lignes, donc on ne va pas utiliser latitude et longitude (absence d'information)
Si j'ai le temps sinon, j'indiquerai un point grossier avec le code commune d'une latitude et longitude pour avoir une météo précise, mais ca c'est si j'ai le temps

In [73]:
#En ile de france
accident_velo_idf.groupby('jour')["identifiant accident"].count()
#Pour ca il faut utiliser les compteurs vélo

jour
0 - lundi       2497
1 - mardi       2798
2 - mercredi    2921
3 - jeudi       2842
4 - vendredi    2683
5 - samedi      1980
6 - dimanche    1820
Name: identifiant accident, dtype: int64

In [88]:
accident_velo_idf_meteo['luminosite'][accident_velo_idf_meteo['luminosite'] == 'Plein jour'] = 'Jour'
accident_velo_idf_meteo['luminosite'][accident_velo_idf_meteo['luminosite'] == 'Nuit avec éclairage public allumé'] = 'Nuit'
accident_velo_idf_meteo['luminosite'][accident_velo_idf_meteo['luminosite'] == 'Nuit avec éclairage public non allumé'] = 'Nuit'
accident_velo_idf_meteo['luminosite'][accident_velo_idf_meteo['luminosite'] == 'Nuit sans éclairage public'] = 'Nuit'
accident_velo_idf_meteo['luminosite'][accident_velo_idf_meteo['luminosite'] == 'Crépuscule ou aube'] = 'Nuit'

C:\Users\Vinciane\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\Vinciane\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Vinciane\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel packag

In [445]:
accident_velo_idf_meteo.groupby('luminosite')["identifiant accident"].count()

luminosite
Jour    13727
Nuit     3814
Name: identifiant accident, dtype: int64

In [446]:
accident_velo_idf_meteo['conditions atmosperiques'][accident_velo_idf_meteo['conditions atmosperiques'] == 'Normale'] = 'Normale'
accident_velo_idf_meteo['conditions atmosperiques'][accident_velo_idf_meteo['conditions atmosperiques'] == 'Temps éblouissant'] = 'Normale'
accident_velo_idf_meteo['conditions atmosperiques'][accident_velo_idf_meteo['conditions atmosperiques'] == 'Brouillard - fumée'] = 'Couvert'
accident_velo_idf_meteo['conditions atmosperiques'][accident_velo_idf_meteo['conditions atmosperiques'] == 'Temps couvert'] = 'Couvert'
accident_velo_idf_meteo['conditions atmosperiques'][accident_velo_idf_meteo['conditions atmosperiques'] == 'Pluie forte'] = 'Precipitations'
accident_velo_idf_meteo['conditions atmosperiques'][accident_velo_idf_meteo['conditions atmosperiques'] == 'Pluie légère'] = 'Precipitations'
accident_velo_idf_meteo['conditions atmosperiques'][accident_velo_idf_meteo['conditions atmosperiques'] == 'Neige - grêle'] = 'Precipitations'
accident_velo_idf_meteo['conditions atmosperiques'][accident_velo_idf_meteo['conditions atmosperiques'] == 'Vent fort - tempête'] = 'Precipitations'
accident_velo_idf_meteo['conditions atmosperiques'][accident_velo_idf_meteo['conditions atmosperiques'] == 'Autre'] = 'Normale'
accident_velo_idf_meteo

C:\Users\Vinciane\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\Vinciane\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Vinciane\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel packag

,identifiant accident,date,mois,jour,heure,departement,luminosite,conditions atmosperiques,etat surface,sexe,age,gravite accident,commune
58,200500001606,2005-01-13,01 - janvier,3 - jeudi,11.0,78,Jour,Normale,normale,M,61-62,2 - Blessé hospitalisé,78230
61,200500001930,2005-01-15,01 - janvier,5 - samedi,15.0,95,Jour,Normale,normale,M,58-59,0 - Indemne,95042
221,200500004591,2005-01-04,01 - janvier,1 - mardi,17.0,75,Nuit,Normale,normale,M,36-37,1 - Blessé léger,75001
222,200500004597,2005-01-03,01 - janvier,0 - lundi,9.0,75,Jour,Normale,normale,M,26-27,0 - Indemne,75005
223,200500004600,2005-01-03,01 - janvier,0 - lundi,17.0,75,Nuit,Normale,normale,M,31-32,0 - Indemne,75019
...,...,...,...,...,...,...,...,...,...,...,...,...,...
65531,201800050291,2018-07-10,07 - juillet,1 - mardi,9.0,75,Jour,Normale,normale,F,58-59,2 - Blessé hospitalisé,75101
65532,201800050337,2018-11-17,11 - novembre,5 - samedi,12.0,75,Jour,Normale,normale,M,59-60,1 - Blessé léger,75101
65533,201800050355,2018-02-01,02 - février,3 - jeudi,15.0,75,Jour,Couvert,mouillée,M,31-32,1 - Blessé léger,75102
65534,201800050359,2018-02-12,02 - février,0 - lundi,9.0,75,Jour,Normale,normale,F,29-30,1 - Blessé léger,75101


In [484]:
accident_velo_idf_meteo.groupby('conditions atmosperiques')["identifiant accident"].count()

conditions atmosperiques
Couvert             646
Normale           15137
Precipitations     1758
Name: identifiant accident, dtype: int64

### Les heures de lever et de coucher du soleil : 

In [453]:
import time
import datetime
#On regarde l'heure et la date de la requête
time.strftime('%Y-%m-%d %H:%M:%S', time.localtime()) #Date et heure en entier
Date = time.strftime('%Y-%m-%d', time.localtime())
Time = time.strftime('%H:%M', time.localtime()) #Heure uniquement
print('Voic la date de la recherche : ', Date)
print("Voici l'heure de la recherche :", Time)

Voic la date de la recherche :  2020-12-08
Voici l'heure de la recherche : 21:50


Difficulté pour trouver une base de donnée propre : seulement à télécharger par année

On prend la base de donnée sur le site : http://ptaff.ca/soleil/?l1pays=France&l1etat=Paris&l1ville=Paris&l2pays=&l2etat=&l1cityname=Paris%2C+Paris%2C+France&l1ltd=48&l1ltm=51&l1lts=0&l1ltx=N&l1lgd=2&l1lgm=19&l1lgs=59&l1lgx=E&l1tz=1.0&l1dst=EU&l2cityname=&l2ltd=&l2ltm=&l2lts=&l2ltx=N&l2lgd=&l2lgm=&l2lgs=&l2lgx=E&l2tz=0&l2dst=&year=2020&month=12&day=08&lang=fr_CA&go=Voir+le+graphe%21
Le but, ce serait de mettre ce fichier csv sur Github

In [475]:
heures_soleil = pd.read_csv('D:/Données/ENSAE/lever_coucher_soleil.csv',  sep = ',')
list(heures_soleil)

['Date', ' Lever du soleil', ' Coucher du soleil', ' Durée du jour']

In [476]:
heures_soleil.loc[heures_soleil['Date'] == Date] 
# On obtient bien, les heures de lever et de coucher du soleil pour le jour demandé

,Date,Lever du soleil,Coucher du soleil,Durée du jour
342,2020-12-08,08:31,16:54,08:23


In [477]:
heures_soleil.loc[heures_soleil['Date'] == Date][' Coucher du soleil']

342     16:54
Name:  Coucher du soleil, dtype: object

In [481]:
heures_soleil.loc[heures_soleil['Date'] == Date]
if  Time >=heures_soleil.loc[heures_soleil['Date'] == Date][' Lever du soleil'].any() and Time <=heures_soleil.loc[heures_soleil['Date'] == Date][' Coucher du soleil'].any():
    print('Vous roulez de jour')
else: 
    print('Attention, vous roulez de nuit')

Attention, vous roulez de nuit


## A partir de la, inutile : 